## 自动求导

假设我们想对函数$y = 2 \pmb x^\top \pmb x$ 关于列向量$\pmb x$求导

In [2]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

### 在我们计算$y$关于$\pmb x$的梯度之前，我们需要一个地方存储梯度。

In [5]:
# 等价与x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
x.grad # 默认值是None

### 计算$y$

In [6]:
# 内积
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

### 通过调用反向传播函数来自动计算$y$关于$x$每个分量的梯度

In [7]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [8]:
x.grad == 4 * x

tensor([True, True, True, True])

### 计算$x$的另一个函数

In [9]:
# 默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

其中，向量$\pmb x$求和相当于向量$\pmb x$乘一个单位向量$\pmb e$，那么，$y$对$x$求导后，$y^{'} = \pmb e$ 